In [0]:
%run /Repos/brucejenks@live.com/databricks-pipelines/pipeline1_batch_delta/utils/utils_upsert_with_hashstring


In [0]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()

# Load raw vendor registry (output from ADF)
df_raw = spark.read.format("parquet").load("dbfs:/mnt/adf-silver")



# Select and clean necessary fields
df_clean = df_raw.select(
    "vendor_id", "industry", "headquarters", "onwatchlist",
    "registrationdate", "tier", "ingestion_timestamp", "source_tag"
)

# Define target cleaned Delta table
target_path = "/mnt/delta/silver/vendor_registry_clean"

# Define deduplication key
unique_keys = ["vendor_id"]  # Can add registrationdate or other compound key if needed

from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, target_path):
    DeltaTable.forPath(spark, target_path).toDF().unpersist()
    dbutils.fs.rm(target_path, recurse=True)
    print(f"Dropped old target at {target_path}")

# Upsert using utility
upsert_with_hashstring(
    spark=spark,
    df=df_clean,
    target_path=target_path,
    unique_cols=unique_keys,
    hash_col_name="hashstring",
    verbose=True
)
